<a href="https://colab.research.google.com/github/odeandialamsyah/nlp_zero_to_hero/blob/main/NyobaBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# --- 1. Muat Data dari CSV ---
file_csv = 'data_teks_dummy.csv'
df = pd.read_csv(file_csv)
print(f"Data berhasil dimuat dari '{file_csv}'. Total {len(df)} baris.")
print(df.head())
print("-" * 50)

Data berhasil dimuat dari 'data_teks_dummy.csv'. Total 15 baris.
                                                teks      label
0  Cuaca hari ini sangat cerah dan menyenangkan, ...    positif
1  Harga minyak dunia melonjak tajam, memicu kekh...    ekonomi
2  Pertandingan final Liga Champions semalam bera...   olahraga
3  Peluncuran produk smartphone terbaru mendapatk...  teknologi
4  Pemerintah mengumumkan kebijakan baru untuk me...    politik
--------------------------------------------------


In [4]:
# --- 2. Encoding Label ---
# BERT memerlukan label dalam bentuk numerik. Kita akan mengkonversi 'positif', 'ekonomi', dll.
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])
num_labels = len(label_encoder.classes_) # Jumlah kategori unik
print(f"Label unik: {label_encoder.classes_}")
print(f"Jumlah label: {num_labels}")
print(df.head())
print("-" * 50)

Label unik: ['ekonomi' 'kesehatan' 'olahraga' 'politik' 'positif' 'teknologi']
Jumlah label: 6
                                                teks      label  label_encoded
0  Cuaca hari ini sangat cerah dan menyenangkan, ...    positif              4
1  Harga minyak dunia melonjak tajam, memicu kekh...    ekonomi              0
2  Pertandingan final Liga Champions semalam bera...   olahraga              2
3  Peluncuran produk smartphone terbaru mendapatk...  teknologi              5
4  Pemerintah mengumumkan kebijakan baru untuk me...    politik              3
--------------------------------------------------


In [6]:
# --- 3. Split Data (Training dan Testing) ---
# Membagi data menjadi data pelatihan (80%) dan data pengujian (20%)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['teks'].tolist(),
    df['label_encoded'].tolist(),
    test_size=0.2, # 20% untuk pengujian
    random_state=42 # Untuk hasil yang bisa direproduksi
    # Removed stratify because the least populated class has only 1 member
)
print(f"Jumlah data pelatihan: {len(train_texts)}")
print(f"Jumlah data pengujian: {len(val_texts)}")
print("-" * 50)

Jumlah data pelatihan: 12
Jumlah data pengujian: 3
--------------------------------------------------


In [9]:
# --- 4. Tokenisasi dengan BERT Tokenizer ---
# Menggunakan tokenizer yang sesuai dengan model BERT
# 'indobenchmark/indobert-base-uncased' adalah pilihan bagus untuk Bahasa Indonesia
# Jika ingin pakai model multi-bahasa, bisa 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained('cahya/bert-base-indonesian-522M')

# Fungsi untuk tokenisasi data
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

print("\n--- Contoh Hasil Tokenisasi (Data Pelatihan Pertama) ---")
print("Input IDs (representasi numerik token):")
print(train_encodings['input_ids'][0])
print("Attention Mask (mengidentifikasi token aktual vs padding):")
print(train_encodings['attention_mask'][0])
print("-" * 50)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]


--- Contoh Hasil Tokenisasi (Data Pelatihan Pertama) ---
Input IDs (representasi numerik token):
tensor([    3,  2630,  1542,  5555, 13349,  5036,  2283,  7534,  5219,  2242,
        22270,    17,     1,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,  

In [10]:
# --- 5. Membuat Dataset Khusus untuk PyTorch/Transformers ---
# Ini adalah wrapper agar data bisa digunakan oleh Trainer
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

In [11]:
# --- 6. Memuat Model BERT untuk Klasifikasi ---
# 'indobert-base-uncased' adalah model BERT pre-trained untuk Bahasa Indonesia
# num_labels harus sesuai dengan jumlah kategori unik yang kita miliki
model = BertForSequenceClassification.from_pretrained('cahya/bert-base-indonesian-522M', num_labels=num_labels)
print("\nModel BERT berhasil dimuat.")
print("-" * 50)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]


Model BERT berhasil dimuat.
--------------------------------------------------


In [13]:
# --- 7. Konfigurasi Pelatihan ---
# TrainingArguments adalah kelas dari Hugging Face untuk mengkonfigurasi pelatihan
training_args = TrainingArguments(
    output_dir='./results',          # Direktori output untuk checkpoint model dan log
    num_train_epochs=3,              # Jumlah epoch pelatihan
    per_device_train_batch_size=8,   # Ukuran batch per GPU/CPU untuk pelatihan
    per_device_eval_batch_size=8,    # Ukuran batch per GPU/CPU untuk evaluasi
    warmup_steps=500,                # Jumlah langkah untuk pemanasan laju pembelajaran
    weight_decay=0.01,               # L2 regularization
    logging_dir='./logs',            # Direktori untuk log TensorBoard
    logging_steps=100,
    eval_strategy="epoch",     # Evaluasi setelah setiap epoch
    save_strategy="epoch",           # Simpan model setelah setiap epoch
    load_best_model_at_end=True,     # Muat model terbaik (berdasarkan metrik evaluasi) di akhir pelatihan
    metric_for_best_model="accuracy",# Metrik yang digunakan untuk menentukan model terbaik
    report_to="none"                 # Jangan laporkan ke platform logging eksternal (misal: Weights & Biases)
)

In [14]:
# --- 8. Fungsi Komputasi Metrik Evaluasi ---
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [15]:
# --- 9. Inisialisasi Trainer dan Latih Model ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("\n--- Memulai Fine-tuning Model BERT ---")
trainer.train()
print("\n--- Fine-tuning Selesai! ---")
print("-" * 50)


--- Memulai Fine-tuning Model BERT ---


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.712117,0.333333,0.111111,0.333333,0.166667
2,No log,1.713045,0.333333,0.111111,0.333333,0.166667
3,No log,1.715060,0.333333,0.111111,0.333333,0.166667



--- Fine-tuning Selesai! ---
--------------------------------------------------


In [17]:
# --- 10. Evaluasi Akhir ---
print("\n--- Evaluasi Model di Data Pengujian ---")
eval_results = trainer.evaluate()
print(f"Hasil Evaluasi Akhir: {eval_results}")
print("-" * 50)


--- Evaluasi Model di Data Pengujian ---


Hasil Evaluasi Akhir: {'eval_loss': 1.7121171951293945, 'eval_accuracy': 0.3333333333333333, 'eval_precision': 0.1111111111111111, 'eval_recall': 0.3333333333333333, 'eval_f1': 0.16666666666666666, 'eval_runtime': 1.4511, 'eval_samples_per_second': 2.067, 'eval_steps_per_second': 0.689, 'epoch': 3.0}
--------------------------------------------------


In [18]:
# --- 11. Contoh Prediksi pada Teks Baru ---
print("\n--- Contoh Prediksi pada Teks Baru ---")
teks_baru_1 = "Kualitas pelayanan di rumah sakit ini perlu ditingkatkan."
teks_baru_2 = "Startup baru ini mendapat investasi besar dari investor asing."

new_texts = [teks_baru_1, teks_baru_2]
new_encodings = tokenize_function(new_texts)

# Prediksi menggunakan model yang sudah dilatih
model.eval() # Set model ke mode evaluasi
with torch.no_grad(): # Nonaktifkan perhitungan gradient untuk inferensi
    outputs = model(input_ids=new_encodings['input_ids'],
                    attention_mask=new_encodings['attention_mask'])
    logits = outputs.logits # Dapatkan output logit
    predictions = torch.argmax(logits, dim=-1) # Ambil indeks dengan probabilitas tertinggi

# Konversi kembali ID label ke nama label asli
predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

print(f"Teks: '{teks_baru_1}' -> Prediksi: '{predicted_labels[0]}'")
print(f"Teks: '{teks_baru_2}' -> Prediksi: '{predicted_labels[1]}'")
print("-" * 50)


--- Contoh Prediksi pada Teks Baru ---
Teks: 'Kualitas pelayanan di rumah sakit ini perlu ditingkatkan.' -> Prediksi: 'politik'
Teks: 'Startup baru ini mendapat investasi besar dari investor asing.' -> Prediksi: 'politik'
--------------------------------------------------
